# 02 Huia Experience Training

# Setup 
## Install Tensorflow 2 Nightly and other Libraries

In [ ]:
#tf addons only works with alpha for now

#!pip install tensorflow-gpu==2.0.0-alpha0 
#!pip install tensorflowjs=2.0.0-alpha0

#!pip install fastai=1.0.52
#!pip install tensorflow-addons
#!pip install opencv-python
#!pip install scipy
#!pip install sklearn

#!pip install tf-nightly-gpu-2.0-preview==2.0.0.dev20190305 --force-reinstall 
#!pip install tensorflow-gpu==2.0.0-alpha0  --force-reinstall
#!pip install tensorflowjs==1.0.1 --force-reinstall

#!pip install tf-nightly-gpu-2.0-preview --upgrade --force-reinstall
#!pip install pathlib
#!pip install matplotlib
!pip freeze | egrep 'tensor|tb|tf|numpy'

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras
#import tensorflowjs as tfjs
import pathlib
import os
import random
#import tensorflow_addons as tfa # not used for now as it is incompatible with tf.data.Dataset

# enable logging to make sure we are running on the GPU
#tf.debugging.set_log_device_placement(True)

# check tensorflow version
tf.__version__

In [ ]:

# clear any active session
tf.keras.backend.clear_session()

AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 48

# Data
We have to understand well our data as this is fundamental to achieve good results.

In [ ]:
root_path = pathlib.Path("./training_data/")
image_path = root_path /"images"
json_path = root_path /"json"
augmented_path = root_path/"augmented_imgs"

image_path,json_path ,augmented_path

In [ ]:
# lets set the random seed so we can reproduce our results
random.seed(7)

# get filenames from directories 
all_image_paths = list(image_path.glob('*.png')) + list(augmented_path.glob('*.png'))
#all_json_paths = list(json_path.glob('*.json'))

all_image_paths = [str(path) for path in all_image_paths] # convert to strings
random.shuffle(all_image_paths) # randomize

In [ ]:
len(all_image_paths)#, len(all_json_paths)

In [ ]:
all_image_paths[:20] 

In [ ]:
import re
# extract categories for classification
pat = r'/([^/]+)_\d+.png$'
all_image_labels = [str(re.search(pat,str(image)).group(1)).lower() for image in all_image_paths]
len(all_image_labels)

In [ ]:
import IPython.display as display
import matplotlib.pyplot as plt

# lets check samples of our images to see what they look like
for n in range(3):  
    image = random.choice(all_image_paths)
    display.display(display.Image(str(image)))
    print(f"file: {image}")

In [ ]:
# get unique classes
huia_person=[]
for label in all_image_labels:
    if label not in huia_person:
        huia_person.append(label)
huia_person = sorted(huia_person) # sort label list
huia_person

In [ ]:
# put them in a dict for lookup
label_to_index = dict((name, index) for index,name in enumerate(huia_person))
label_to_index

In [ ]:
# lets format it, so we can copy and paste the dict direclty into javascript :-)
print("POSE_CLASSES = {")
for index,name in enumerate(huia_person):
    print("\t" +str(index)+": '"+name+"',")
print("}")

In [ ]:
# load data into tf data
img_raw = tf.io.read_file(all_image_paths[0])
img_tensor = tf.image.decode_image(img_raw)
print(img_tensor.shape)
print(img_tensor.dtype)
plt.imshow(img_tensor)

#img_raw.numpy()




In [ ]:
import numpy
from random import randint
import math

#@tf.function
def  preprocess_image(image):
    #print("EagerMode:" + str(tf.executing_eagerly()))
    # decode PNG
    image = tf.image.decode_png(image, channels=3)    
    
    # data augmentation - doing statically via fastai, as tensorflow addons is not compatible with graph mode yet 
    # rotate random
    #degrees = random.randint(-6,6)
    #angle = degrees * math.pi / 180
    #image = tfa.image.rotate(image,angle,interpolation='BILINEAR')
    # comented as this is not supported in graph mode yet
    
    # random crop - images are tensors of shape (500,640,3)
    #crop_factor = (random.randint(0,20)/100) # generate numbers between 0.7 and 1.0
    #new_width = int(640 * (1-crop_factor))
    #new_height = int(500 * (1-crop_factor))
    # resize_image_with_crop_or_pad
    # image = tf.image.resize_image_with_crop_or_pad(image,new_height,new_width)
    # commented as we are doing prepocessing on static files
    
    # resize
    image = tf.image.resize(image, [224, 224])

    # normalize = convert to [-1:1]    
    offset = 127.5
    image = (image-offset)/offset
    return image

#@tf.function
def load_and_preprocess_image(path):    
    image = tf.io.read_file(path)
    return preprocess_image(image)


In [ ]:
# load data into tf data
img_tensor = load_and_preprocess_image(all_image_paths[0])
plt.imshow(img_tensor)

In [ ]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
path_ds

next(iter(path_ds))

In [ ]:
# force non performatic eager mode as tfa.image doesn't support graph mode yet
#image_ds = path_ds.map(lambda path: tf.py_function(func=load_and_preprocess_image,inp=[path],Tout=tf.float32))
image_ds = path_ds.map(map_func=load_and_preprocess_image,num_parallel_calls = AUTOTUNE).cache(filename='images_normalized')

In [ ]:
all_image_labels_idx = [label_to_index[label] for label in all_image_labels]
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels_idx, tf.int64))
len(all_image_labels)

In [ ]:
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
image_label_ds
image_count = len(all_image_labels)

type(image_label_ds)

In [ ]:
ds = image_label_ds.cache()
ds = ds.apply(tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
ds = ds.batch(BATCH_SIZE).prefetch(buffer_size=AUTOTUNE)
ds

In [ ]:
# we will use a pretrained mobilenet for transferlearning
mobilenet = tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')
mobilenet.trainable = False
mobilenet.summary()

In [ ]:
image_batch, label_batch = next(iter(ds))
print(image_batch.shape)

In [ ]:
label_batch[0].numpy()
image_batch[0].numpy()

In [ ]:
image_batch, label_batch = next(iter(ds))

feature_map_batch = mobilenet(image_batch)
print(feature_map_batch.shape)

In [ ]:
model = tf.keras.Sequential([
    mobilenet,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1024,activation='relu',bias_initializer=tf.keras.initializers.he_normal()),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1024,activation='relu',bias_initializer=tf.keras.initializers.he_normal()),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(512,activation='relu',bias_initializer=tf.keras.initializers.he_normal(),name='features'),
    tf.keras.layers.Dense(len(huia_person),activation='softmax'),
])
model.summary()






In [ ]:
# class HuiaModel(tf.keras.Model):
#     def __init__(self):
#         super(HuiaModel,self).__init__()
#         self.mobilenet = mobilenet
#         self.globalavg = tf.keras.layers.GlobalAveragePooling2D()
#         self.drop1 = tf.keras.layers.Dropout(0.3)
#         self.dense1 = tf.keras.layers.Dense(1024,activation='relu',bias_initializer=tf.keras.initializers.he_normal())
#         self.drop2 = tf.keras.layers.Dropout(0.3)
#         self.dense2 = tf.keras.layers.Dense(1024,activation='relu',bias_initializer=tf.keras.initializers.he_normal())
#         self.drop3 = tf.keras.layers.Dropout(0.2)
#         self.dense3 = tf.keras.layers.Dense(512,activation='relu',bias_initializer=tf.keras.initializers.he_normal(),name='features')
#         self.softmax = tf.keras.layers.Dense(len(huia_person),activation='softmax')
    
#     def call(self,x):
#         x = self.mobilenet(x)
#         x = self.globalavg(x)
#         x = self.drop1(x)
#         x = self.dense1(x)
#         x = self.drop2(x)
#         x = self.dense2(x)
#         x = self.drop3(x)
#         x = self.dense3(x)
#         return self.softmax(x)

# max_lr = 3e-4

# model = HuiaModel()
# loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
# optimizer = tf.keras.optimizers.Adam(lr=max_lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-07, amsgrad=False)

# train_loss = tf.keras.metrics.Mean(name='train_loss')
# train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

# test_loss = tf.keras.metrics.Mean(name='test_loss')
# test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')


# @tf.function
# def train_step(images, labels):
#   with tf.GradientTape() as tape:
#     predictions = model(images)
#     loss = loss_object(labels, predictions)
#   gradients = tape.gradient(loss, model.trainable_variables)
#   optimizer.apply_gradients(zip(gradients, model.trainable_variables))

#   train_loss(loss)
#   train_accuracy(labels, predictions)
    
# @tf.function
# def test_step(images, labels):
#   predictions = model(images)
#   t_loss = loss_object(labels, predictions)

#   test_loss(t_loss)
#   test_accuracy(labels, predictions)
    
    
# EPOCHS = 20

# for epoch in range(EPOCHS):
#   for images, labels in train_ds:
#     train_step(images, labels)

#   for test_images, test_labels in test_ds:
#     test_step(test_images, test_labels)

#   template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
#   print (template.format(epoch+1,
#                          train_loss.result(),
#                          train_accuracy.result()*100,
#                          test_loss.result(),
#                          test_accuracy.result()*100))
    
    

In [ ]:
logit_batch = model(image_batch).numpy()

print("min logit:", logit_batch.min())
print("max logit:", logit_batch.max())
print()

print("Shape:", logit_batch.shape)

In [ ]:
len(model.trainable_variables) 
model.summary()

In [ ]:
steps_per_epoch=tf.math.ceil((len(all_image_paths)-838)/BATCH_SIZE).numpy()
steps_per_epoch

In [ ]:
# One Cycle https://www.kaggle.com/robotdreams/one-cycle-policy-with-keras
# import OneCycleLR
from datetime import datetime
now = datetime.now()
log_dir = "./logs/1_" + now.strftime("%Y%m%d-%H%M%S") + "/"

callbacks = []

# logging
tbCallback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)
tbCallback.set_model(model)
callbacks.append(tbCallback)

epochs = 20
max_lr = 3e-4

base_lr = max_lr/10
# max_m = 0.98
# base_m = 0.85

# cyclical_momentum = False
# augment = True
# cycles = 2.35

# iterations = round(len(all_image_paths)/BATCH_SIZE*epochs)
# iterations = list(range(0,iterations+1))
# step_size = len(iterations)/(cycles)

# clr = OneCycleLR.CyclicLR(base_lr=base_lr,
#                 max_lr=max_lr,
#                 step_size=step_size,
#                 max_m=max_m,
#                 base_m=base_m,
#                 cyclical_momentum=cyclical_momentum)

#callbacks.append(clr)

# class myCallback(tf.keras.callbacks.Callback):
#   def on_epoch_end(self, epoch, logs={}):
#     if(logs.get('val_accuracy')>0.9):
#       print("\nReached 60% accuracy so cancelling training!")
#       self.model.stop_training = True

# earlyStopping = tf.keras.callbacks.EarlyStopping(patience=2, monitor='loss')
# callbacks.append(earlyStopping)

# chkPoint = tf.keras.callbacks.ModelCheckpoint('./models.h5')
# chkPoint.set_model(model)
# callbacks.append(chkPoint)

In [ ]:
# skip 20% validation images
test_dataset = ds.take(838) 
train_dataset = ds.skip(838)

In [ ]:
mobilenet.trainable = False
model.compile(optimizer=tf.keras.optimizers.Adam(lr=max_lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-07, amsgrad=False), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, epochs=epochs,steps_per_epoch=steps_per_epoch,verbose=1,validation_data=test_dataset,
          callbacks=callbacks)

In [ ]:
!pip install scikit-learn

In [ ]:
from datetime import datetime
import OneCycleLR
now = datetime.now()
log_dir = "./logs/2_" + now.strftime("%Y%m%d-%H%M%S") + "/"

callbacks = []

epochs = 50
max_lr = 3e-6

base_lr = max_lr/10
max_m = 0.98
base_m = 0.85

cyclical_momentum = False
augment = True
cycles = 2.35

iterations = round(len(all_image_paths)/BATCH_SIZE*epochs)
iterations = list(range(0,iterations+1))
step_size = len(iterations)/(cycles)

clr = OneCycleLR.CyclicLR(base_lr=base_lr,
                max_lr=max_lr,
                step_size=step_size,
                max_m=max_m,
                base_m=base_m,
                cyclical_momentum=cyclical_momentum)

#callbacks.append(clr)

# logging
tbCallback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=1)
tbCallback.set_model(model)
callbacks.append(tbCallback)

In [ ]:
mobilenet.trainable = True
model.compile(optimizer=tf.keras.optimizers.Adam(lr=max_lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-07, amsgrad=False), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_dataset, epochs=epochs,steps_per_epoch=steps_per_epoch,verbose=1,validation_data=test_dataset,
          callbacks=callbacks)

In [ ]:
model.save("models/huia_mob_224_final9.h5") # 1.0

In [ ]:
model

# Convert to TENSORFLOW JS / Quantization

In [6]:
# Tensorflow 2 Alpha has a bug exporting to TFJS, so we need to use a nightly version
import tensorflowjs as tfjs
import numpy as np

# model = tf.keras.models.load_model("models/huia_mob_224_final_one_cycle.h5")
# model.load_weights("models/huia_mob_224_final_one_cycle.h5")

#tfjs.converters.save_keras_model(model, "models_tfjs/tfjs_huia_mob_224_final9_q16",quantization_dtype=np.uint16)
#tfjs.converters.save_keras_model(model, "models_tfjs/tfjs_huia_mob_224_final9_q8",quantization_dtype=np.uint8)

# run on command line, above commands are buggy at the moment
!tensorflowjs_converter \
    --input_format=keras \
    --output_format=tfjs_layers_model \
    ./models/huia_mob_224_final_one_cycle.h5 \
    ./models_tfjs/tfjs_huia_mob_224_final_q16a \
    --quantization_bytes 2


# copy tfjs model to 03_experience/static and change App.vue reference to load it

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model(model,'model.png',show_layer_names=False) #,show_shapes=True)

# Tensorboard Visualization

In [ ]:
# https://www.tensorflow.org/tensorboard/
import tensorboard as tb
%load_ext tensorboard.notebook

#notebook.list() # View open TensorBoard instances
#notebook.display(port=6006, height=1000)
#!kill 25264

In [ ]:
%tensorboard --logdir ./logs

# Test / Predict

In [ ]:
import numpy as np
sample = np.reshape(img_final,[1,224,224,3])

predict = int(model.predict_classes(sample))
predict

In [ ]:
[key for key,value in label_to_index.items() if value == predict]

In [ ]:
label_to_index

# UTILS

## Delete Images that are empty

In [ ]:
# during image capture some images are empty, so we automatically delete them

# open images and delete if they are empty
def remove_empty_imgs(imgpath):
    for item in imgpath.iterdir():
        im = imread(str(item), format='png')
        if np.count_nonzero(im)==0: 
            print(item,np.count_nonzero(im))
            os.remove(str(item))
            
#remove_empty_imgs(image_path)

## Sync JSONs with Images

In [ ]:
# since we might delete unwanted images for training, this will also delete the json files
# sync json & images 

def sync_json_images(json_paths, img_path):
    for json in json_paths:
        #print(json.stem, end=" ")
        img = img_path/f"{json.stem}.png"
        if not Path(img).exists():
            print(f"{img} doesn't exist, deleting {json}")
            Path(json).unlink()
        
#sync_json_images(all_json_paths, image_path)

# Static Data Augmentation
Decided to use FASTAI to statically preprocess data augmentation, as tensorflow addons (0.3.1) still didn't support graph mode and is therefore not compatible with tf.data.Dataset mappings




In [ ]:
from fastai.vision import *
from fastai.metrics import error_rate
from random import randint
import pathlib

root_path = pathlib.Path("./training_data/")
save_fast = Path('./training_data/augmented_imgs/')
save_fast.mkdir(parents=True, exist_ok=True)

tfms = get_transforms(do_flip=False, 
                            flip_vert=False, 
                            max_rotate=6, 
                            max_zoom=1.2, 
                            max_lighting=None, 
                            max_warp=0.2, 
                            p_affine=0.2, 
                            p_lighting=0)


image_path = root_path /"images"
all_image_paths = list(image_path.glob('*.png'))


def generate_augmented(qty):
    for f in all_image_paths:
        image = open_image(f)
        for i in range(0,qty):
            image_fast = image.apply_tfms(tfms[0])
            save_name = str(save_fast) + '/' + f.stem + '99' + str(i) + f.suffix
            print(save_name)
            image_fast.save(save_name)

#generate 10 variations of each image
#generate_augmented(10)